#Setup

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 27.9 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [ ]:
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 12.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
import spacy

#**Make df from sample text**

In [ ]:
nlp =spacy.load('en_core_web_trf')

In [ ]:
%%writefile english_demo_text.txt
"okay there is a flag. and it's young man trying to fly a kite. and this sailboat with two people. and one's waving. and someone's fishing off the pier. and puppy dog. and couple has in front of his house. picnicking in front of his house. and there's a car parked in front of the house. and they're in the garage. there's a tree in front of the house. there's small trees leading down to the way way way out."

Writing english_demo_text.txt


# General Methods

In [ ]:
def filterText(text):
  import nltk
  # this tokeniser takes care of contractions nicely
  from nltk.tokenize import WhitespaceTokenizer
  # Create a reference variable for Class WhitespaceTokenizer
  tk = WhitespaceTokenizer()
  trackWords = tk.tokenize(text)
  #removing unwanted characters, excluding contractions
  bad_chars = [',', ':', '!', '\"', '?']
  filteredTrackWords = [''.join(filter(lambda i: i not in bad_chars, word)) for word in trackWords]
  #remove empty words
  filteredTrackWords = [i for i in filteredTrackWords if i]
  return " ".join(filteredTrackWords)

In [ ]:
f = open("english_demo_text.txt","r")
doc = nlp(filterText(f.read()))

In [ ]:
def setupSpacyDF(doc):
  import pandas as pd
  cols = ("text",  "POS", "STOP", "DEP")
  rows = []
  for t in doc:
    if t.pos_ == 'PUNCT':
      #not considering punctuation
      continue
    row = [t.text, t.pos_, t.is_stop, t.dep_]
    rows.append(row)
  df = pd.DataFrame(rows, columns=cols)
  return df

In [ ]:
print(list(setupSpacyDF(doc)['text']))

['okay', 'there', 'is', 'a', 'flag', 'and', 'it', "'s", 'young', 'man', 'trying', 'to', 'fly', 'a', 'kite', 'and', 'this', 'sailboat', 'with', 'two', 'people', 'and', 'one', "'s", 'waving', 'and', 'someone', "'s", 'fishing', 'off', 'the', 'pier', 'and', 'puppy', 'dog', 'and', 'couple', 'has', 'in', 'front', 'of', 'his', 'house', 'picnicking', 'in', 'front', 'of', 'his', 'house', 'and', 'there', "'s", 'a', 'car', 'parked', 'in', 'front', 'of', 'the', 'house', 'and', 'they', "'re", 'in', 'the', 'garage', 'there', "'s", 'a', 'tree', 'in', 'front', 'of', 'the', 'house', 'there', "'s", 'small', 'trees', 'leading', 'down', 'to', 'the', 'way', 'way', 'way', 'out']


#Easy feats

In [ ]:
def posTagCounts(df, linguisticFeatures):
  # 5.20.22: This line is problematic
  #tag_types = list(df['POS'].unique())
  #solution: scroll to bottom of https://stackoverflow.com/questions/58215855/how-to-get-full-list-of-pos-tag-and-dep-in-spacy
  tag_types = ["ADJ", "ADP", "ADV", "AUX", "CONJ", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "SCONJ", "SYM", "VERB", "X", "SPACE"]
  for category in tag_types:
    tag_description = "POS_TAG:" + category
    linguisticFeatures[tag_description] = (df['POS'] == category).sum()

In [ ]:
def openClosedRatio(df, linguisticFeatures):
  open_class = {'VERB', 'NOUN', 'ADJ', 'ADV'}
  tag_types = list(df['POS'].unique())
  open_words = 0
  closed_words = 0
  for category in tag_types:
    if category in open_class:
      open_words += (df['POS'] == category).sum()
    else:
      closed_words += (df['POS'] == category).sum()
  ratio = open_words/closed_words if closed_words else "NaN"
  linguisticFeatures["Open to Closed Words Ratio"] = ratio
# CHECK EDGE CASE


POS tags are consistent across languages

In [ ]:
def posNormalizedCounts(df, linguisticFeatures):
  #tag_types = ["ADJ", "ADP", "ADV", "AUX", "CONJ", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "SCONJ", "SYM", "VERB", "X", "SPACE"]
  filtered_tag_types = ["ADJ", "ADP", "ADV", "AUX","CONJ", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "SCONJ", "VERB"]
  #for category in tag_types:
  #DECIDE
  for category in filtered_tag_types:
    tag_description = "POS_TAG:" + category
    linguisticFeatures[tag_description] = (df['POS'] == category).sum()/df.shape[0]

#Dependency tags

In [ ]:
#source: https://spacy.io/models/en#en_core_web_trf-labels
englishDepTags = ['ROOT', 'acl', 'acomp','advcl','advmod','agent','amod','appos','attr','aux','auxpass','case','cc','ccomp','compound','conj','csubj','csubjpass','dative','dep','det','dobj','expl','intj','mark','meta','neg','nmod','npadvmod','nsubj','nsubjpass','nummod','oprd','parataxis','pcomp','pobj','poss','preconj','predet','prep','prt','punct','quantmod','relcl','xcomp']

In [ ]:
#source: https://spacy.io/models/zh#zh_core_web_trf-labels
chineseDepTags = ["ROOT", "acl", "advcl:loc", "advmod", "advmod:dvp", "advmod:loc","advmod:rcomp", "amod", "amod:ormod", "appos", "aux:asp", "aux:ba", "aux:modal", "aux:prtmod", "auxpass", "case", "cc", "ccomp", "compound:nn", "compound:vc", "conj", "cop", "dep", "det", "discourse", "dobj", "etc","mark","mark:clf", "name", "neg","nmod","nmod:assmod","nmod:poss","nmod:prep", "nmod:range", "nmod:tmod", "nmod:topic", "nsubj", "nsubj:xsubj", "nsubjpass","nummod", "parataxis:prnmod", "punct", "xcomp"]

In [ ]:
advmodSpecificChineseTags = ["advmod:dvp", "advmod:loc","advmod:rcomp"]
amodSpecificChineseTags = ["amod:ormod"]
auxSpecificChineseTags = ["aux:asp", "aux:ba", "aux:modal", "aux:prtmod","auxpass"]
compoundSpecificChineseTags = ["compound:nn", "compound:vc"]
markSpecificChineseTags = ["mark:clf"]
nmodSpecificChineseTags = ["nmod:assmod","nmod:poss","nmod:prep", "nmod:range", "nmod:tmod", "nmod:topic"]
nsubjSpecificChineseTags = ["nsubj:xsubj"]
nummodSpecificChineseTags = ["nummod"]

In [ ]:
chineseSimplifiedDepTags = []
for tag in chineseDepTags:
  if tag in advmodSpecificChineseTags:
    chineseSimplifiedDepTags.append("advmod")
  elif tag in amodSpecificChineseTags:
    chineseSimplifiedDepTags.append("amod")
  elif tag in auxSpecificChineseTags:
    chineseSimplifiedDepTags.append("aux")
  elif tag in compoundSpecificChineseTags:
    chineseSimplifiedDepTags.append("compound")
  elif tag in markSpecificChineseTags:
    chineseSimplifiedDepTags.append("mark")
  elif tag in nmodSpecificChineseTags:
    chineseSimplifiedDepTags.append("nmod")
  elif tag in nsubjSpecificChineseTags:
    chineseSimplifiedDepTags.append("nsubj")
  elif tag in nummodSpecificChineseTags:
    chineseSimplifiedDepTags.append("nummod")
  else:
    chineseSimplifiedDepTags.append(tag)

In [ ]:
chineseSimplifiedDepTags = set(chineseSimplifiedDepTags)
print(sorted(chineseSimplifiedDepTags))

['ROOT', 'acl', 'advcl:loc', 'advmod', 'amod', 'appos', 'aux', 'case', 'cc', 'ccomp', 'compound', 'conj', 'cop', 'dep', 'det', 'discourse', 'dobj', 'etc', 'mark', 'name', 'neg', 'nmod', 'nsubj', 'nsubjpass', 'nummod', 'parataxis:prnmod', 'punct', 'xcomp']


In [ ]:
print(sorted(englishDepTags))

['ROOT', 'acl', 'acomp', 'advcl', 'advmod', 'agent', 'amod', 'appos', 'attr', 'aux', 'auxpass', 'case', 'cc', 'ccomp', 'compound', 'conj', 'csubj', 'csubjpass', 'dative', 'dep', 'det', 'dobj', 'expl', 'intj', 'mark', 'meta', 'neg', 'nmod', 'npadvmod', 'nsubj', 'nsubjpass', 'nummod', 'oprd', 'parataxis', 'pcomp', 'pobj', 'poss', 'preconj', 'predet', 'prep', 'prt', 'punct', 'quantmod', 'relcl', 'xcomp']


In [ ]:
overlapChineseEnglishDepTags = chineseSimplifiedDepTags.intersection(set(englishDepTags))
print(overlapChineseEnglishDepTags)

{'neg', 'mark', 'ROOT', 'cc', 'aux', 'nmod', 'nsubj', 'ccomp', 'nsubjpass', 'advmod', 'acl', 'punct', 'nummod', 'det', 'compound', 'xcomp', 'conj', 'amod', 'appos', 'dobj', 'dep', 'case'}


In [ ]:
def depTagCounts(df, linguisticFeatures):
  overlapChineseEnglishDepTags = chineseSimplifiedDepTags.intersection(set(englishDepTags))
  #English:tag_types = ["acl", "acomp", "advcl", "advmod", "agent", "amod", "appos", "attr", "aux", "auxpass", "case", "cc", "ccomp", "compound", "conj", "cop", "csubj", "csubjpass", "dative", "dep", "det", "dobj", "expl", "intj", "mark", "meta", "neg", "nn", "npmod", "nsubj", "nsubjpass", "oprd", "obj", "obl", "pcomp", "pobj", "poss", "preconj", "prep", "prt", "punct",  "quantmod", "relcl", "root", "xcomp"]
  for category in overlapChineseEnglishDepTags:
    depTagDescription = "GrammarRelation:" + category
    linguisticFeatures[depTagDescription] = (df['DEP'] == category).sum()/df.shape[0]

In [ ]:
complexTagsInEnglishGCI = ['csubj', 'csubjpass', 'ccomp', 'xcomp', 'acomp', 'pobj', 'advcl', 'mark', 'acl', 'nounmod', 'complm', 'infmod', 'partmod', 'nmod']

In [ ]:
chineseSimplifiedDepTags.intersection(set(complexTagsInEnglishGCI))

{'acl', 'ccomp', 'mark', 'nmod', 'xcomp'}

In [ ]:
overlapChineseEnglishDepTags.intersection(set(complexTagsInEnglishGCI))

{'acl', 'ccomp', 'mark', 'nmod', 'xcomp'}

In [ ]:
def grammarComplexityIndex(df, linguisticFeatures):
  complexTags = chineseSimplifiedDepTags.intersection(set(complexTagsInEnglishGCI))
  numComplexTags = 0
  for tag in complexTags:
    numComplexTags += (df['DEP'] == tag).sum()
  totalNumTags = len(df['DEP'])
  grammarComplexity = numComplexTags/totalNumTags
  linguisticFeatures['Grammar Complexity Index'] = grammarComplexity

#propositional density

In [ ]:
def propositionalDensity(df, linguisticFeatures):
  pdTags = ['VERB', 'ADJ', 'ADV', 'ADP', 'CONJ', 'CCONJ', 'SCONJ']
  numPDTags = 0
  for tag in pdTags:
    numPDTags += (df['POS'] == tag).sum()
  totalNumPDTags = len(df['POS'])
  propositionalDensity = numPDTags/totalNumPDTags
  linguisticFeatures['Propositional Density'] = propositionalDensity

#Type token ration

In [ ]:
def typeTokenRatio(df, linguisticFeatures):
  numTokens = len(df)
  numTypes = len(set(df['text'].str.lower()))
  linguisticFeatures["# of unique words"] = numTypes
  typeTokenRatio = numTypes/numTokens
  linguisticFeatures["Type Token Ratio"] = typeTokenRatio

In [ ]:
def mov_avg_type_token_ratio(df, window_size=50):
  sum_type_token_ratio = 0
  numWindows = 0
  window_start = 0
  window_end = window_size
  while(window_end <= len(df)):
    currentWindow = df['text'][window_start:window_end]
    numTypes = len(set(df['text'][window_start:window_end].str.lower()))
    window_type_token_ratio = numTypes/window_size
    sum_type_token_ratio += window_type_token_ratio
  #update variables for next iteration
    numWindows +=1
    window_start +=1
    window_end +=1
  if numWindows == 0:
    return pd.NA
  moving_average_type_token_ratio = sum_type_token_ratio/numWindows
  return moving_average_type_token_ratio

In [ ]:
def determine_repetition(currentWord, words):
  for word in words:
      if (word != currentWord):
        return False
  return True

In [ ]:
def repetition_of_word_n(n,df):
  numRepetitions = 0
  first_word = df['text'][0]
  current_word = first_word
  current_word_index = 0
  next_window_start_index = 1
  next_window_end_index = n
  next_words = None
  while(next_window_end_index < len(df)):
    nextWords = list(df['text'][next_window_start_index:next_window_end_index])
    #print(nextWords)
    if (determine_repetition(current_word, nextWords)):
      numRepetitions += 1
    #update for next iteration
    current_word = nextWords[0]
    current_word_index +=1
    next_window_start_index +=1
    next_window_end_index +=1
  return numRepetitions

#Stopwords

In [ ]:
!pip install stopwordsiso


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.1 MB/s eta 0:00:00


In [ ]:
import stopwordsiso
from stopwordsiso import stopwords
englishStopWords = stopwords("en")  # Chinese
print(englishStopWords)
len(englishStopWords)

{'appropriate', 'such', 'young', 'mustnt', 'looks', 'kr', 'whod', 'probably', 'theyd', 'instead', 'tm', 'below', 'definitely', "there's", 'shes', "shan't", 'invention', 'someday', 'orders', 'tell', 'actually', 'copy', 'let', 'cn', 'ag', 'top', 'approximately', 'just', 'till', 'co', 'do', '10', "that've", 'yes', 'de', "how'll", 'works', 'seeing', 'ee', 'oughtnt', "'ll", 'sees', 'wants', 'htm', 'ug', 'themselves', 'consequently', 'getting', "what've", "don't", 'above', 'towards', 'beings', 'where', 'doubtful', 'line', 'along', 'keys', 'na', 'uz', 'notwithstanding', 'twenty', 'face', 'thin', 'aint', 'sj', 'these', 'lc', 'eleven', 'evenly', 'proud', 'uucp', 'could', 'ge', 'grouped', 'like', 'tr', 'er', 'novel', 'immediate', 'tz', 'with', 'mrs', 'mean', 'resulted', 'gh', 'li', 'contains', "mightn't", 'vu', 'ye', 'pl', 'ae', 'ma', 'ableabout', 'higher', 'one', 'forever', 'their', 'text', 'similarly', 'second', 'x', 'v', 'fify', 'http', 'can', 'found', 'thered', 'parts', 'gb', 'very', 'regard

1298

In [ ]:
[len(stopword) for stopword in englishStopWords]

In [ ]:
englishStopWords = stopwords("en")
print(englishStopWords)
len(englishStopWords)

{'appropriate', 'such', 'young', 'mustnt', 'looks', 'kr', 'whod', 'probably', 'theyd', 'instead', 'tm', 'below', 'definitely', "there's", 'shes', "shan't", 'invention', 'someday', 'orders', 'tell', 'actually', 'copy', 'let', 'cn', 'ag', 'top', 'approximately', 'just', 'till', 'co', 'do', '10', "that've", 'yes', 'de', "how'll", 'works', 'seeing', 'ee', 'oughtnt', "'ll", 'sees', 'wants', 'htm', 'ug', 'themselves', 'consequently', 'getting', "what've", "don't", 'above', 'towards', 'beings', 'where', 'doubtful', 'line', 'along', 'keys', 'na', 'uz', 'notwithstanding', 'twenty', 'face', 'thin', 'aint', 'sj', 'these', 'lc', 'eleven', 'evenly', 'proud', 'uucp', 'could', 'ge', 'grouped', 'like', 'tr', 'er', 'novel', 'immediate', 'tz', 'with', 'mrs', 'mean', 'resulted', 'gh', 'li', 'contains', "mightn't", 'vu', 'ye', 'pl', 'ae', 'ma', 'ableabout', 'higher', 'one', 'forever', 'their', 'text', 'similarly', 'second', 'x', 'v', 'fify', 'http', 'can', 'found', 'thered', 'parts', 'gb', 'very', 'regard

1298

Stopwords: words that are commonly used in an language (https://kavita-ganesan.com/what-are-stop-words/)

In [ ]:
def numStopWords(df, stopWords=englishStopWords):
  result = 0
  for word in list(df['text']):
    if word in stopWords:
      result+=1
  return result/len(df)

In [ ]:
def averageSentenceLength(df, doc, linguisticFeatures):
  numSentences = len(list(doc.sents))
  #numWords = len(df)
  #both implementations achieve same result for demo sample
  averageSentenceLength = len(df)/numSentences#numSentences/len(df)

  # for sent in doc.sents:
  #   numWords += len(sent)
  #avgSentenceLength = numWords/numSentences
  linguisticFeatures['# of words'] = len(df['POS'])#, corrected on 6.25.2022
  #linguisticFeatures['# of words'] = numWords
  linguisticFeatures["Average sentence length"] = averageSentenceLength

#**Word Length by Morphemes**

In [ ]:
!pip install polyglot
#!sudo apt-get install python-numpy libicu-dev
!pip install pyicu #problematic on terminal: use conda install -c conda-forge pyicu
!pip install pycld2
!pip install morfessor
from polyglot.downloader import downloader
from polyglot.text import Text, Word
!polyglot download morph2.en

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52558 sha256=9e081748226dcdee141e3cd356083f213aec1600453455e63a5dd5f6e37093c9
  Stored in directory: /root/.cache/pip/wheels/aa/92/4a/b172589446ba537db3bdb9a1f2204f27fe71217981c14ac368
Successfully built polyglot
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyicu: filename=PyICU-2.12-cp310-cp310-linux_x86_64.whl size=1754547 sha256=cc35e62ea7b27b4873a1e1f2f4ae536464224d22f01e3eb470b3a78b88aa3a0b
  Stored in directory: /root/.cache/pip/wheels/74/60/95/66d97ac2fdc8be8e526c4254047405fe77feaf064282d1ad07
Successfully built pyicu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 49.1 MB/s eta 

In [ ]:
def wordLengthByMorphemes(df,linguisticFeatures):
  wordsLengthByMophemes = [len(Word(word, language="en").morphemes) for word in list(df['text'])]
  linguisticFeatures["Word Length by Morphemes Mean"] = np.mean(wordsLengthByMophemes)
  linguisticFeatures["Word Length by Morphemes Standard Deviation"] = np.std(wordsLengthByMophemes)
  #linguisticFeatures["Max Word Length by Morphemes"] = np.max(wordsLengthByMophemes)
  #linguisticFeatures["Min Word Length by Morphemes"] = np.min(wordsLengthByMophemes)

#Word length by phoneme

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

model = T5ForConditionalGeneration.from_pretrained('charsiu/g2p_multilingual_byT5_small_100') # 'charsiu/g2p_multilingual_byT5_tiny_16_layers_100'

tokenizer = AutoTokenizer.from_pretrained('google/byt5-small')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

In [ ]:
# tokenized English words
words = ['Char', 'siu', 'is', 'a', 'Cantonese', 'style', 'of', 'barbecued', 'pork']
words = ['<eng-us>: '+i for i in words]
out = tokenizer(words,padding=True,add_special_tokens=False,return_tensors='pt')
preds = model.generate(**out,num_beams=1,max_length=50) # We do not find beam search helpful. Greedy decoding is enough.
phones = tokenizer.batch_decode(preds.tolist(),skip_special_tokens=True)
print(phones)
# Output: ['ˈtʃɑɹ', 'ˈsiw', 'ˈɪs', 'ˈɑ', 'ˈkæntəˌniz', 'ˈstaɪɫ', 'ˈɑf', 'ˈbɑɹbɪkˌjud', 'ˈpɔɹk']


['ˈtʃɑɹ', 'ˈʃu', 'ˈɪs', 'ˈeɪ', 'ˌkæntəˈniz', 'ˈstaɪɫ', 'ˈəf', 'ˈbɑɹbɪkˌjud', 'ˈpɔɹk']


In [ ]:
def wordLengthByPhonemes(df,linguisticFeatures):
  words = list(df['text'])
  words = ['<eng-us>: '+i for i in words]
  out = tokenizer(words,padding=True,add_special_tokens=False,return_tensors='pt')
  preds = model.generate(**out,num_beams=1,max_length=50) # We do not find beam search helpful. Greedy decoding is enough.
  phones = tokenizer.batch_decode(preds.tolist(),skip_special_tokens=True)
  linguisticFeatures["Word Length by Phonemes Mean"] = np.mean([len(i) for i in phones])
  linguisticFeatures["Word Length by Phonemes Standard Deviation"] = np.std([len(i) for i in phones])
  #linguisticFeatures["Max Word Length by Morphemes"] = np.max(wordsLengthByMophemes)
  #linguisticFeatures["Min Word Length by Morphemes"] = np.min(wordsLengthByMophemes)

# New Section

#**Word Frequency**

In [ ]:
import pandas as pd

In [ ]:
wordFreqDF = pd.read_csv('SUBTLEX-EN-WF.csv')
wordFreqDF.head()

,Word,FREQcount,CDcount,FREQlow,Cdlow,SUBTLWF,Lg10WF,SUBTLCD,Lg10CD
0,the,1501908,8388,1339811,8388,29449.18,6.1766,100.00,3.9237
1,to,1156570,8383,1138435,8380,22677.84,6.0632,99.94,3.9235
2,a,1041179,8382,976941,8380,20415.27,6.0175,99.93,3.9234
3,you,2134713,8381,1595028,8376,41857.12,6.3293,99.92,3.9233
4,and,682780,8379,515365,8374,13387.84,5.8343,99.89,3.9232


In [ ]:
englishWordFreqProbs = dict(zip(wordFreqDF['Word'], wordFreqDF['Lg10WF']))

In [ ]:
englishWordContextualDiversity = dict(zip(wordFreqDF['Word'], wordFreqDF['Lg10CD']))

In [ ]:
def wordFreqMetrics(df,linguisticFeatures):
  openClassPOSTags = ['NOUN', 'VERB', 'ADJ', 'ADV']
  openClassDF = df[df.POS.isin(openClassPOSTags)]
  wordFreqVals = []
  wordCDVals = []
  for word in list(openClassDF['text']):
    wordFreqVal = englishWordFreqProbs.get(word)
    if wordFreqVal != None:
      wordFreqVals.append(wordFreqVal)
    wordCD = englishWordContextualDiversity.get(word)
    if wordCD != None:
      wordCDVals.append(wordCD)
  linguisticFeatures["Word Frequency Mean"] = np.nanmean(wordFreqVals)
  linguisticFeatures["Word Frequency Standard Deviation"] = np.nanstd(wordFreqVals)
  linguisticFeatures["Word Contextual Diversity Mean"] = np.nanmean(wordCDVals)
  linguisticFeatures["Word Contextual Diversity Standard Deviation"] = np.nanstd(wordCDVals)

#**Age of Acquisition**

In [ ]:
import pandas as pd

Source: https://link.springer.com/article/10.3758/s13428-020-01455-8

In [ ]:
ageOfAcquistionDF = pd.read_csv('AoA_ratings_Kuperman_et_al_BRM.csv')
ageOfAcquistionDF.head()

,Word,OccurTotal,OccurNum,Freq_pm,Rating.Mean,Rating.SD,Dunno
0,a,22.0,22.0,20415.27,2.89,1.21,1.00
1,aardvark,18.0,18.0,0.41,9.89,3.66,1.00
2,abacus,20.0,13.0,0.24,8.69,3.77,0.65
3,abalone,18.0,13.0,0.51,12.23,3.54,0.72
4,abandon,19.0,19.0,8.10,8.32,2.75,1.00


In [ ]:
ageOfAcquistionMeanMapping = dict(zip(list(ageOfAcquistionDF['Word']), list(ageOfAcquistionDF['Rating.Mean'])))

In [ ]:
ageOfAcquistionSDMapping = dict(zip(list(ageOfAcquistionDF['Word']), list(ageOfAcquistionDF['Rating.SD'])))

In [ ]:
def compute_AoA_MappingForWordList(df,linguisticFeatures):
  wordsAOAMeanList = []
  wordsAOASDList = []
  openClassPOSTags = ['NOUN', 'VERB', 'ADJ', 'ADV']
  openClassDF = df[df.POS.isin(openClassPOSTags)]
  for word in list(openClassDF['text']):
    wordMeanAOA = ageOfAcquistionMeanMapping.get(word.lower())
    if wordMeanAOA == None:
      wordsAOAMeanList.append(np.nan)
      continue
    wordsAOAMeanList.append(float(wordMeanAOA))
    wordAOASD = ageOfAcquistionSDMapping.get(word.lower())
    if wordAOASD == None:
      wordsAOASDList.append(np.nan)
      continue
    wordsAOASDList.append(float(wordAOASD))
  linguisticFeatures["Age of Acquistion Mean Average"] = np.nanmean(wordsAOAMeanList)
  linguisticFeatures["Age of Acquistion SD Average"] = np.nanmean(wordsAOASDList)

#Yngve and frazier

In [ ]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 933.2/933.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 10.9 MB/s eta 0:00:00


In [ ]:
import stanza

In [ ]:
stanza.download('en') # download Chinese simplified
pipe = stanza.Pipeline(lang='en', processors='tokenize,  pos, lemma, depparse, constituency') # initialize Chinese neural pipeline

INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| mwt          | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [ ]:
def get_yngve_score(TreeNode, score, leaf_num, total_score):
  if TreeNode.is_leaf():
    total_score += score
  else:
    num_children = len(TreeNode.children)
    for i in range(0, num_children):
      total_score = get_yngve_score(TreeNode.children[i], score + (num_children - 1) - i, i, total_score)
  return total_score

In [ ]:
def get_frazier_score(node, my_score, total_score, sent_parent):
  if node.is_leaf():
    sent_val = 1.5 if sent_parent else 0
    #print("Leaf Node: ", node, " Gets Score: ", my_score + sent_val)
    total_score = total_score + my_score + sent_val
  else:
    num_children = len(node.children)
    for i in range(0, num_children):
      sent_bool = True if (node.label == "S" and i == 0) or sent_parent else False
      sibling_num = 0 if ((i == (num_children - 1)) and (num_children != 1)) else 1
      total_score = get_frazier_score(node.children[i], sibling_num, total_score, sent_bool)
  return total_score

In [ ]:
def yngve_fraz_score_normalized(text,linguisticFeatures):
  numWords = 0
  textYngveScore = 0
  textFrazierScore = 0
  info_map = {}
  for sent in text.sentences:
    stripped_sent = [word.text for word in sent.words if word.upos != "PUNCT"]
    preprocessed = ' '.join(stripped_sent)
    if len(pipe(preprocessed).sentences) ==0:
      continue
    clean_sent = pipe(preprocessed).sentences[0];
    sent_tree_root = clean_sent.constituency.children[0]
    score = 0
    yngve_score = get_yngve_score(sent_tree_root, score, 0, total_score=0)
    #print("Yngve score:", yngve_score)
    fraz_score = get_frazier_score(sent_tree_root, 0, total_score=0, sent_parent=False)
    #print("Frazier score:", fraz_score)
    info_map[sent.text] = [clean_sent.constituency, yngve_score, fraz_score]
    textYngveScore += yngve_score
    textFrazierScore += fraz_score
    numWords += len(stripped_sent)
  linguisticFeatures["Yngve"] = textYngveScore/numWords
  linguisticFeatures["Frazier"] = textFrazierScore/numWords

#Run everything

In [ ]:
def extractLinguisticFeatures(transcription, sampleName):
  import spacy
  nlp =spacy.load('en_core_web_trf')
  doc = nlp(filterText(transcription))
  df = setupSpacyDF(doc)
  linguisticFeatures = {}
  posNormalizedCounts(df, linguisticFeatures)
  linguisticFeatures['Noun to Verb ratio'] = (df['POS'] == 'NOUN').sum()/(df['POS'] == 'VERB').sum()
  depTagCounts(df, linguisticFeatures)
  grammarComplexityIndex(df, linguisticFeatures)
  propositionalDensity(df, linguisticFeatures)
  typeTokenRatio(df, linguisticFeatures)
  wordLengthByMorphemes(df,linguisticFeatures)
  wordLengthByPhonemes(df,linguisticFeatures)
  linguisticFeatures["Moving Average Type Token Ratio"] = mov_avg_type_token_ratio(df, window_size=50)
  linguisticFeatures["Repetitions of word twice"] = repetition_of_word_n(2,df)
  linguisticFeatures["StopwordsProportion"] = numStopWords(df)
  averageSentenceLength(df, doc, linguisticFeatures)
  wordFreqMetrics(df,linguisticFeatures)
  compute_AoA_MappingForWordList(df,linguisticFeatures)
  yngve_fraz_score_normalized(pipe(transcription),linguisticFeatures)
  return linguisticFeatures

In [ ]:
f = open("english_demo_text.txt","r")
transcription = f.read()
feats = extractLinguisticFeatures(transcription, 'Demo')
print(len(feats))
feats

59


{'POS_TAG:ADJ': 0.022988505747126436,
 'POS_TAG:ADP': 0.13793103448275862,
 'POS_TAG:ADV': 0.022988505747126436,
 'POS_TAG:AUX': 0.04597701149425287,
 'POS_TAG:CONJ': 0.0,
 'POS_TAG:CCONJ': 0.09195402298850575,
 'POS_TAG:DET': 0.11494252873563218,
 'POS_TAG:INTJ': 0.011494252873563218,
 'POS_TAG:NOUN': 0.27586206896551724,
 'POS_TAG:NUM': 0.011494252873563218,
 'POS_TAG:PART': 0.011494252873563218,
 'POS_TAG:PRON': 0.11494252873563218,
 'POS_TAG:PROPN': 0.0,
 'POS_TAG:SCONJ': 0.0,
 'POS_TAG:VERB': 0.13793103448275862,
 'Noun to Verb ratio': 2.0,
 'GrammarRelation:neg': 0.0,
 'GrammarRelation:mark': 0.0,
 'GrammarRelation:ROOT': 0.13793103448275862,
 'GrammarRelation:cc': 0.09195402298850575,
 'GrammarRelation:aux': 0.034482758620689655,
 'GrammarRelation:nmod': 0.0,
 'GrammarRelation:nsubj': 0.05747126436781609,
 'GrammarRelation:ccomp': 0.0,
 'GrammarRelation:nsubjpass': 0.0,
 'GrammarRelation:advmod': 0.022988505747126436,
 'GrammarRelation:acl': 0.034482758620689655,
 'GrammarRelati

In [ ]:
!unzip taukadialTranscriptions.zip

In [ ]:
import os
inputDirectory = "trimmedEnglishTranscriptions_WhisperLargeV3/"
fileAddresses = sorted(os.listdir(inputDirectory))
linguisticFeats = []
for file in fileAddresses:
  sampleID = file.split('_')[-1].split('.')[0]
  print("Working on", sampleID)
  f = open(inputDirectory+file,"r")
  transcription = f.read()
  fileLinguisticFeats = extractLinguisticFeatures(transcription, sampleID)
  linguisticFeats.append(fileLinguisticFeats)

Working on taukdial-002-1
Working on taukdial-002-2
Working on taukdial-002-3
Working on taukdial-005-1
Working on taukdial-005-2
Working on taukdial-005-3
Working on taukdial-008-1
Working on taukdial-008-2
Working on taukdial-008-3
Working on taukdial-011-1
Working on taukdial-011-2
Working on taukdial-011-3
Working on taukdial-014-1
Working on taukdial-014-2
Working on taukdial-014-3
Working on taukdial-015-1
Working on taukdial-015-2
Working on taukdial-015-3
Working on taukdial-020-1
Working on taukdial-020-2
Working on taukdial-020-3
Working on taukdial-026-1
Working on taukdial-026-2
Working on taukdial-026-3
Working on taukdial-030-1
Working on taukdial-030-2
Working on taukdial-030-3
Working on taukdial-031-1
Working on taukdial-031-2
Working on taukdial-031-3
Working on taukdial-032-1
Working on taukdial-032-2
Working on taukdial-032-3
Working on taukdial-035-1
Working on taukdial-035-2
Working on taukdial-035-3
Working on taukdial-037-1
Working on taukdial-037-2
Working on t

<ipython-input-69-8fbc0b30194a>:8: RuntimeWarning: invalid value encountered in scalar divide
  linguisticFeatures['Noun to Verb ratio'] = (df['POS'] == 'NOUN').sum()/(df['POS'] == 'VERB').sum()
<ipython-input-46-281fbac266cd>:13: RuntimeWarning: Mean of empty slice
  linguisticFeatures["Word Frequency Mean"] = np.nanmean(wordFreqVals)
/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
<ipython-input-46-281fbac266cd>:15: RuntimeWarning: Mean of empty slice
  linguisticFeatures["Word Contextual Diversity Mean"] = np.nanmean(wordCDVals)
<ipython-input-51-fed047322367>:17: RuntimeWarning: Mean of empty slice
  linguisticFeatures["Age of Acquistion Mean Average"] = np.nanmean(wordsAOAMeanList)
<ipython-input-51-fed047322367>:18: RuntimeWarning: Mean of empty slice
  linguisticFeatures["Age of Acquistion SD Average"] = np.nanmean(wordsAOASDList)


Working on taukdial-054-2
Working on taukdial-054-3
Working on taukdial-057-1
Working on taukdial-057-2
Working on taukdial-057-3
Working on taukdial-068-1
Working on taukdial-068-2
Working on taukdial-068-3
Working on taukdial-069-1
Working on taukdial-069-2
Working on taukdial-069-3
Working on taukdial-070-1
Working on taukdial-070-2
Working on taukdial-070-3
Working on taukdial-071-1
Working on taukdial-071-2
Working on taukdial-071-3
Working on taukdial-074-1
Working on taukdial-074-2
Working on taukdial-074-3
Working on taukdial-076-1
Working on taukdial-076-2
Working on taukdial-076-3
Working on taukdial-077-1
Working on taukdial-077-2
Working on taukdial-077-3
Working on taukdial-078-1
Working on taukdial-078-2
Working on taukdial-078-3
Working on taukdial-079-1
Working on taukdial-079-2
Working on taukdial-079-3
Working on taukdial-083-1
Working on taukdial-083-2
Working on taukdial-083-3
Working on taukdial-085-1
Working on taukdial-085-2
Working on taukdial-085-3
Working on t

In [ ]:
fileAddresses = sorted(os.listdir(inputDirectory))
sampleNames = [file.split('_')[-1].split('.')[0] for file in fileAddresses]
sampleNames

In [ ]:
# Convert list of dictionaries to DataFrame
df = pd.DataFrame(linguisticFeats)
df.insert(0, 'SampleID', sampleNames)
# Display the DataFrame
df.head()

,SampleID,POS_TAG:ADJ,POS_TAG:ADP,POS_TAG:ADV,POS_TAG:AUX,POS_TAG:CONJ,POS_TAG:CCONJ,POS_TAG:DET,POS_TAG:INTJ,POS_TAG:NOUN,...,# of words,Average sentence length,Word Frequency Mean,Word Frequency Standard Deviation,Word Contextual Diversity Mean,Word Contextual Diversity Standard Deviation,Age of Acquistion Mean Average,Age of Acquistion SD Average,Yngve,Frazier
0,taukdial-002-1,0.060897,0.105769,0.054487,0.083333,0.0,0.048077,0.147436,0.016026,0.185897,...,312,0.076923,4.185365,0.719286,3.602073,0.381731,4.653962,1.850189,1.711538,1.475962
1,taukdial-002-2,0.024000,0.112000,0.040000,0.072000,0.0,0.064000,0.168000,0.016000,0.232000,...,125,0.064000,3.786963,0.930928,3.299967,0.607039,4.252857,1.717857,1.952000,1.912000
2,taukdial-002-3,0.050000,0.092857,0.021429,0.100000,0.0,0.042857,0.157143,0.028571,0.185714,...,140,0.100000,3.812723,0.975619,3.325153,0.638027,4.453636,1.933636,1.578571,1.825000
3,taukdial-005-1,0.079268,0.079268,0.085366,0.097561,0.0,0.036585,0.115854,0.012195,0.201220,...,164,0.091463,3.906950,0.901416,3.414522,0.613490,4.900484,1.891129,1.585366,1.237805
4,taukdial-005-2,0.038961,0.103896,0.038961,0.077922,0.0,0.051948,0.155844,0.012987,0.220779,...,77,0.051948,3.819650,0.856363,3.331781,0.565775,4.776000,1.857000,1.792208,1.681818


In [ ]:
from google.colab import files
df.to_csv("taukadialEnglish_AllLinguisticFeatures.csv",index=None)
files.download("taukadialEnglish_AllLinguisticFeatures.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#blabla feats

In [ ]:
import stanza

# Download the English model for Stanza
stanza.download('en')

# Initialize the pipeline
nlp = stanza.Pipeline('en', processors='tokenize,mwt,pos,lemma,depparse', use_gpu=False)

# Process text and extract features
def extract_features(text):
    doc = nlp(text)

    inflected_verbs = 0
    auxiliary_verbs = 0
    gerund_verbs = 0
    participles = 0
    clauses = 0
    noun_phrases = 0
    verb_phrases = 0
    infinitive_phrases = 0
    prepositional_phrases = 0
    dependent_clauses = 0
    discourse_markers = 0

    for sent in doc.sentences:
        # Count verbs and their types
        for word in sent.words:
            if 'VB' in word.upos:  # VB: Verb, base form
                inflected_verbs += 1
            elif 'VBD' in word.upos or 'VBN' in word.upos:  # VBD: Verb, past tense, VBN: Verb, past participle
                participles += 1
            elif 'VBG' in word.upos:  # VBG: Verb, gerund or present participle
                gerund_verbs += 1
            elif 'MD' in word.upos:  # MD: Modal auxiliary
                auxiliary_verbs += 1

            # Count discourse markers
            if word.deprel == 'discourse':
                discourse_markers += 1

        # Count clauses
        clauses += len(sent.dependencies)

        # Count noun phrases, verb phrases, infinitive phrases, prepositional phrases, and dependent clauses
        for token in sent.tokens:
            if token.misc is not None:
                if 'NP' in token.misc:
                    noun_phrases += 1
                elif 'VP' in token.misc:
                    verb_phrases += 1
                elif 'VBG' in token.misc:  # VB or VBG might indicate infinitive phrase
                    infinitive_phrases += 1
                elif 'PP' in token.misc:
                    prepositional_phrases += 1
                elif 'SBAR' in token.misc:
                    dependent_clauses += 1

    total_tokens = sum(len(sent.words) for sent in doc.sentences)
    clause_rate = clauses / total_tokens
    noun_phrase_rate = noun_phrases / total_tokens
    verb_phrase_rate = verb_phrases / total_tokens
    infinitive_phrase_rate = infinitive_phrases / total_tokens
    prepositional_phrase_rate = prepositional_phrases / total_tokens
    dependent_clause_rate = dependent_clauses / total_tokens
    discourse_marker_rate = discourse_markers / total_tokens

    features = {
        'Proportion of inflected verbs': inflected_verbs / total_tokens,
        'Proportion of auxiliary verbs': auxiliary_verbs / total_tokens,
        'Proportion of gerund verbs': gerund_verbs / total_tokens,
        'Proportion of participles': participles / total_tokens,
        'Number of clauses': clauses,
        'Clause rate': clause_rate,
        'Noun phrase rate': noun_phrase_rate,
        'Verb phrase rate': verb_phrase_rate,
        'Infinitive phrase rate': infinitive_phrase_rate,
        'Prepositional phrase rate': prepositional_phrase_rate,
        'Dependent clause rate': dependent_clause_rate,
        'Discourse marker rate': discourse_marker_rate
    }

    return features

f = open("english_demo_text.txt","r")
text = f.read()

# Extract features
features = extract_features(text)

# Print features
for feature, value in features.items():
    print(f'{feature}: {value}')


INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


Proportion of inflected verbs: 0.0
Proportion of auxiliary verbs: 0.0
Proportion of gerund verbs: 0.0
Proportion of participles: 0.0
Number of clauses: 101
Clause rate: 1.0
Noun phrase rate: 0.0
Verb phrase rate: 0.0
Infinitive phrase rate: 0.0
Prepositional phrase rate: 0.0
Dependent clause rate: 0.0
Discourse marker rate: 0.009900990099009901
